In [ ]:
import tensorflow as tf
import numpy as np
import os

# Set dataset path
ucf_dir = "/path/to/ucf101"

# Load list of class names
with open(os.path.join(ucf_dir, "classInd.txt")) as f:
    class_names = [line.strip().split()[1] for line in f.readlines()]

# Load training data
train_data = []
train_labels = []
for class_id, class_name in enumerate(class_names):
    class_dir = os.path.join(ucf_dir, class_name)
    for filename in os.listdir(class_dir):
        filepath = os.path.join(class_dir, filename)
        video = np.load(filepath)
        train_data.append(video)
        train_labels.append(class_id)

# Load testing data
test_data = []
test_labels = []
with open(os.path.join(ucf_dir, "testlist01.txt")) as f:
    for line in f.readlines():
        filename = line.strip().split()[0]
        class_name = filename.split("/")[0]
        class_id = class_names.index(class_name)
        filepath = os.path.join(ucf_dir, filename)
        video = np.load(filepath)
        test_data.append(video)
        test_labels.append(class_id)

# Convert data to tensors and normalize pixel values to range [0, 1]
train_data = tf.convert_to_tensor(train_data, dtype=tf.float32) / 255.
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_data = tf.convert_to_tensor(test_data, dtype=tf.float32) / 255.
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# Create dataset objects
train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(len(train_data)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(32)

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', input_shape=(16, 112, 112, 3)),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(256, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=10)

# Evaluate the model on the test set
model.evaluate(test_ds)
